In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 4, ndecoderlayers = 2, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1):
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints, 2], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints, 2)
        )
        
        self.distance_matrix = (self.points.unsqueeze(1) - self.points.unsqueeze(2)).square().sum(-1).sqrt() # [batchsize * nsamples, npoints, npoints]
        
        self.previous_point = None
        
        self.points_mask = torch.zeros(
                    [self.batchsize, npoints], dtype=torch.bool, device=device
                )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert list(point_index.size()) == [self.batchsize]
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        if self.previous_point != None:
            self.cost += self.distance_matrix.gather(2, self.previous_point.unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, point_index.unsqueeze(1)).squeeze(1)
        
        self.previous_point = point_index
        self.points_mask.scatter_(1, point_index.unsqueeze(1), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
        
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    
    def laststep(self):
        
        assert self.points_sequence.size(1) == self.npoints
        
        self.cost += self.distance_matrix.gather(2, self.points_sequence[:, 0].unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, self.points_sequence[:, -1].unsqueeze(1)).squeeze(1)
    

env = environment()


def evaluate(epochs = 10, npoints = 10, batchsize = 100, nsamples = 2, negative_cutoff = 1):
    NN.eval()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item())


def train(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        
        NN.eval()
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.15
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 2
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( 
        #        - logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
        #        * positive_reinforcement_binary 
        #        + logprob.view(batchsize, nsamples)[:, :-1] 
        #        * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 40) 
        #        * negative_reinforcement_binary 
        #).mean()
        """
        scaled binary reinforcement
        """
        #loss = ( 
        #        (- logprob.view(batchsize, nsamples)[:, :-1] 
        #        * positive_reinforcement_binary).sum() / (positive_reinforcement_binary.sum() + 1)
        #        + (logprob.view(batchsize, nsamples)[:, :-1] 
        #        * negative_reinforcement_binary).sum() / (negative_reinforcement_binary.sum() + 1)
        #)
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
   




In [ ]:
train(300000, 100, 150, 8, 1)

39.91866683959961 -359.578125 -363.64190673828125 -363.62933349609375 -364.5697021484375 -364.5697021484375
39.580223083496094 -359.28717041015625 -363.615234375 -364.0981750488281 -363.5884704589844 -363.5884704589844
38.601295471191406 -359.0276794433594 -363.60791015625 -363.52984619140625 -363.3526916503906 -363.3526611328125
37.86245346069336 -358.8288879394531 -363.5812072753906 -363.4959716796875 -363.06866455078125 -363.06866455078125
37.37076950073242 -358.74554443359375 -363.5845642089844 -363.82952880859375 -364.19287109375 -364.1927795410156
37.05512619018555 -358.52203369140625 -363.59600830078125 -363.58099365234375 -362.9840087890625 -362.9840393066406
36.70586395263672 -358.35791015625 -363.5384521484375 -363.5024108886719 -364.2391662597656 -364.2391662597656
36.08876419067383 -358.2226257324219 -363.5922546386719 -364.0548095703125 -363.31005859375 -363.3101806640625
35.008216857910156 -357.9819030761719 -363.5354309082031 -363.13458251953125 -363.793701171875 -363.79

29.520553588867188 -340.7972412109375 -360.2201232910156 -360.93011474609375 -360.47247314453125 -360.47247314453125
29.42629623413086 -340.5270690917969 -360.1692810058594 -361.8943786621094 -357.7851257324219 -357.7850341796875
29.66310691833496 -339.8491516113281 -359.8805236816406 -357.72454833984375 -357.613525390625 -357.61358642578125
29.468339920043945 -338.9604797363281 -359.4239196777344 -361.3760070800781 -360.91180419921875 -360.9118347167969
29.428874969482422 -338.50531005859375 -359.3586120605469 -360.733154296875 -361.9049987792969 -361.9049987792969
29.431621551513672 -337.803955078125 -359.3260498046875 -357.19384765625 -357.9802551269531 -357.9802551269531
29.510202407836914 -337.10614013671875 -358.9825134277344 -358.272705078125 -367.13787841796875 -367.1378173828125
29.58261489868164 -336.2461242675781 -358.4915771484375 -360.52325439453125 -355.8010559082031 -355.8011169433594
29.43085289001465 -335.6657409667969 -358.3760986328125 -356.959228515625 -359.02606201

27.038936614990234 -256.60968017578125 -301.1111755371094 -296.9637756347656 -297.2406921386719 -297.24072265625
26.954626083374023 -255.7877655029297 -300.241943359375 -305.4603271484375 -296.7396240234375 -296.7395935058594
26.891752243041992 -254.56187438964844 -299.2150573730469 -303.5511779785156 -297.1541442871094 -297.1541748046875
26.67031478881836 -254.08522033691406 -298.8457946777344 -295.54046630859375 -304.1075744628906 -304.107666015625
26.8212947845459 -253.36370849609375 -298.21246337890625 -299.1794128417969 -291.77459716796875 -291.7746276855469
26.611207962036133 -252.52899169921875 -297.3067321777344 -300.14678955078125 -291.9747009277344 -291.9747009277344
26.181777954101562 -251.69886779785156 -296.4909973144531 -299.35882568359375 -296.88330078125 -296.8833312988281
25.909372329711914 -251.19979858398438 -295.80218505859375 -294.15460205078125 -294.4538879394531 -294.4539489746094
26.106830596923828 -250.55714416503906 -295.28631591796875 -296.15557861328125 -287

17.6766414642334 -224.1123046875 -277.78924560546875 -269.98394775390625 -273.8209228515625 -273.82098388671875
17.5498104095459 -224.1096954345703 -278.2262878417969 -279.02880859375 -293.7029724121094 -293.7029724121094
17.623340606689453 -224.77432250976562 -279.5973815917969 -273.1903381347656 -283.4382019042969 -283.4382629394531
17.17998504638672 -224.66677856445312 -279.81231689453125 -275.0920104980469 -282.4391174316406 -282.4391174316406
17.035694122314453 -224.9401092529297 -280.1404724121094 -285.6016845703125 -281.02685546875 -281.0268859863281
16.807802200317383 -225.1940155029297 -280.7604675292969 -278.968994140625 -287.97314453125 -287.9732360839844
16.853700637817383 -224.78897094726562 -281.2866516113281 -280.181884765625 -274.78179931640625 -274.78179931640625
16.613454818725586 -224.40591430664062 -280.7724914550781 -292.3081359863281 -291.99786376953125 -291.9978942871094
16.559040069580078 -223.88339233398438 -280.1341552734375 -273.78759765625 -286.4225463867187

14.211858749389648 -193.89877319335938 -256.4211730957031 -265.3383483886719 -247.10792541503906 -247.1079559326172
14.166873931884766 -193.78509521484375 -256.35516357421875 -268.0032043457031 -244.60447692871094 -244.60455322265625
14.28030014038086 -194.0296173095703 -256.5082702636719 -265.1614685058594 -267.7979736328125 -267.79803466796875
14.241127014160156 -194.2984161376953 -256.8813781738281 -249.62228393554688 -251.6427001953125 -251.6426544189453
14.179240226745605 -194.07278442382812 -257.4215393066406 -249.39634704589844 -257.6915588378906 -257.6915588378906
14.185254096984863 -194.6548309326172 -257.12506103515625 -263.89508056640625 -266.62982177734375 -266.6297912597656
14.003117561340332 -194.77305603027344 -257.21197509765625 -256.6552734375 -265.65496826171875 -265.65509033203125
14.10222053527832 -194.43727111816406 -257.7615966796875 -257.1122741699219 -242.322998046875 -242.322998046875
13.939720153808594 -194.76356506347656 -257.7475280761719 -253.33302307128906

...

9.657126426696777 -63.86361312866211 -111.20963287353516 -103.81690979003906 -89.95066833496094 -89.95065307617188
9.636168479919434 -64.21333312988281 -111.31559753417969 -94.44281005859375 -106.67657470703125 -106.67658233642578
9.631498336791992 -64.28804016113281 -111.53048706054688 -112.69047546386719 -122.89472961425781 -122.8947525024414
9.615589141845703 -64.42169189453125 -111.40390014648438 -118.38253784179688 -95.72722625732422 -95.7272720336914
9.625588417053223 -63.93852233886719 -110.34834289550781 -120.43230438232422 -108.16604614257812 -108.16604614257812
9.59981918334961 -63.8355598449707 -111.04885864257812 -120.68675231933594 -94.89962768554688 -94.89964294433594
9.593592643737793 -64.09196472167969 -111.1881103515625 -101.54953002929688 -113.56263732910156 -113.56261444091797
9.564007759094238 -63.715576171875 -110.53250885009766 -112.29937744140625 -97.23515319824219 -97.23516845703125
9.579682350158691 -63.835391998291016 -110.80003356933594 -104.98939514160156 -1

9.504348754882812 -63.44066619873047 -110.64685821533203 -99.95831298828125 -113.54402160644531 -113.54400634765625
9.528732299804688 -63.99789047241211 -111.05268096923828 -107.33143615722656 -114.63778686523438 -114.63778686523438
9.548927307128906 -63.737281799316406 -110.9434585571289 -120.32427978515625 -129.61962890625 -129.61964416503906
9.581498146057129 -63.67511749267578 -111.07939147949219 -101.51103210449219 -108.97560119628906 -108.9756088256836
9.545622825622559 -63.84133529663086 -111.3211898803711 -112.74259948730469 -114.51629638671875 -114.51631927490234
9.533324241638184 -63.62968826293945 -110.98316955566406 -107.0091552734375 -105.95502471923828 -105.95504760742188
9.580314636230469 -63.97707748413086 -110.81769561767578 -98.5161361694336 -121.45050048828125 -121.45050048828125
9.573711395263672 -63.64944839477539 -110.66008758544922 -106.66699981689453 -107.55305480957031 -107.55305480957031
9.524649620056152 -63.50968933105469 -110.54148864746094 -99.268783569335

9.526247024536133 -62.007789611816406 -107.90092468261719 -100.98694610595703 -98.95289611816406 -98.9529037475586
9.510403633117676 -61.554935455322266 -106.97258758544922 -102.29863739013672 -98.82051086425781 -98.82051086425781
9.504009246826172 -61.92378234863281 -108.2010269165039 -99.79852294921875 -115.04072570800781 -115.04072570800781
9.571206092834473 -62.008392333984375 -107.73503112792969 -116.7313003540039 -102.88778686523438 -102.88780975341797
9.53592300415039 -61.94633102416992 -108.30233764648438 -95.60564422607422 -96.77299499511719 -96.77301788330078
9.53166675567627 -61.92905044555664 -107.56999969482422 -100.86564636230469 -110.75790405273438 -110.7579116821289
9.491643905639648 -62.2520637512207 -108.45446014404297 -109.70209503173828 -122.15650177001953 -122.15650177001953
9.495891571044922 -62.1462516784668 -108.75178527832031 -98.06393432617188 -104.73999786376953 -104.73999786376953
9.544859886169434 -62.277103424072266 -108.69387817382812 -101.90090942382812 

9.421429634094238 -61.540000915527344 -107.78572845458984 -102.05770874023438 -110.18209838867188 -110.18212890625
9.395424842834473 -62.001930236816406 -108.20867156982422 -106.1850814819336 -98.3582534790039 -98.35824584960938
9.417695045471191 -62.71440124511719 -108.547119140625 -114.50300598144531 -106.94396209716797 -106.94395446777344
9.497272491455078 -62.657997131347656 -109.20624542236328 -115.8300552368164 -116.65994262695312 -116.65994262695312
9.41430377960205 -62.43772888183594 -108.76022338867188 -121.54580688476562 -109.15861511230469 -109.15862274169922
9.419501304626465 -62.2747917175293 -109.19143676757812 -108.99409484863281 -129.2166748046875 -129.21669006347656
9.427061080932617 -62.98131561279297 -109.1081314086914 -122.14893341064453 -120.4422607421875 -120.44226837158203
9.40310287475586 -62.40352249145508 -108.876220703125 -107.64883422851562 -87.18185424804688 -87.1818618774414
9.372925758361816 -63.123497009277344 -108.90467071533203 -106.14308166503906 -103

9.405522346496582 -61.094547271728516 -107.14508819580078 -101.1002426147461 -115.64457702636719 -115.64456176757812
9.417521476745605 -61.4854621887207 -106.81205749511719 -118.38880920410156 -99.72927856445312 -99.72927856445312
9.425180435180664 -61.259674072265625 -106.57125091552734 -95.402099609375 -102.39930725097656 -102.39930725097656
9.363082885742188 -61.27541732788086 -107.2058334350586 -108.59584045410156 -87.83531188964844 -87.8353271484375
9.464666366577148 -61.56204605102539 -107.19600677490234 -89.05319213867188 -114.68010711669922 -114.68013000488281
9.440763473510742 -60.96367263793945 -106.60124206542969 -89.98523712158203 -130.85516357421875 -130.85513305664062
9.425219535827637 -60.892398834228516 -106.12675476074219 -119.36795806884766 -99.9924545288086 -99.99248504638672
9.484871864318848 -60.50295639038086 -105.53565979003906 -109.30699157714844 -115.42365264892578 -115.42365264892578
9.435296058654785 -60.32783508300781 -105.66732025146484 -109.48727416992188 

In [ ]:
train(300000, 100, 100, 8, 1)

In [7]:
eval(10, 20, 100)

3.9546682834625244 -0.4706580638885498
3.9321422576904297 -0.4440091550350189
3.9317967891693115 -0.33326441049575806
3.926325798034668 -0.36147579550743103
3.9361000061035156 -0.40054047107696533
3.966549873352051 -0.35143959522247314
3.996304512023926 -0.40718939900398254
3.9640331268310547 -0.36856964230537415
3.977106809616089 -0.4640328586101532
3.972842216491699 -0.45662763714790344
